# Create `no_snow.nc` File for `sss150` Domain

Create a netCDF4 file containing 1 variable named `snow`.
The coordinates of the variable are `y` and `x` and their sizes are 826 and 710.
The value of `snow` at all points in the domain is floating point zero.

The resulting `no_snow.nc` file can be used as an annual climatology in NEMO atmospheric forcing
that does not require on-the-fly interpolation.
It imposes a no snow, ever condition on the `sss150` NEMO configuration.

In [1]:
import datetime

import numpy
import xarray

Coordinates:

In [2]:
domain_size = (826, 710)  # y, x

y_indices = numpy.arange(domain_size[0], dtype=numpy.int64)
y_coord = xarray.DataArray(
    name="y",
    data=y_indices,
    coords={"y": y_indices},
    attrs={
        "long_name": "Grid Y",
        "standard_name": "y",
        "units": "count",
        "comment": "y values are grid indices in the model y-direction",
    },
)

x_indices = numpy.arange(domain_size[1], dtype=numpy.int64)
x_coord = xarray.DataArray(
    name="x",
    data=x_indices,
    coords={"x": x_indices},
    attrs={
        "long_name": "Grid X",
        "standard_name": "x",
        "units": "count",
        "comment": "x values are grid indices in the model x-direction",
    },
)

`snow` variable:

In [3]:
snow_array = xarray.DataArray(
    name="snow",
    data=numpy.zeros(domain_size, dtype=numpy.float32),
    coords={"y": y_coord, "x": x_coord},
    attrs={
        "long_name": "Snow Precipitation Rate",
        "standard_name": "snowfall_flux",
        "units": "kg m-2 s-1",
        
    }
)

In [4]:
snow_array

<xarray.DataArray 'snow' (y: 826, x: 710)> Size: 2MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * y        (y) int64 7kB 0 1 2 3 4 5 6 7 8 ... 818 819 820 821 822 823 824 825
  * x        (x) int64 6kB 0 1 2 3 4 5 6 7 8 ... 702 703 704 705 706 707 708 709
Attributes:
    long_name:      Snow Precipitation Rate
    standard_name:  snowfall_flux
    units:          kg m-2 s-1

Dataset:

In [5]:
no_snow_ds = xarray.Dataset(
    data_vars={"snow": snow_array},
    coords={"y": y_coord, "x": x_coord},
    attrs={
        "conventions": "CF-1.6",
        "title": "sss150 NEMO domain Yearly Snowfall Climatology",
        "institution": "Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia",
        "source": "https://github.com/SalishSeaCast/analysis-doug/blob/main/notebooks/sss150/NoSnow.ipynb",
        "comment": "Annual climatology HRDPS-like atmospheric forcing file for NEMO that always supplies zero as the solid precipitation rate value.",
        "history": f"{datetime.datetime.now():%Y-%m-%d %H:%M}: Created.",
    },
)

no_snow_ds

<xarray.Dataset> Size: 2MB
Dimensions:  (y: 826, x: 710)
Coordinates:
  * y        (y) int64 7kB 0 1 2 3 4 5 6 7 8 ... 818 819 820 821 822 823 824 825
  * x        (x) int64 6kB 0 1 2 3 4 5 6 7 8 ... 702 703 704 705 706 707 708 709
Data variables:
    snow     (y, x) float32 2MB 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    conventions:  CF-1.6
    title:        sss150 NEMO domain Yearly Snowfall Climatology
    institution:  Dept of Earth, Ocean & Atmospheric Sciences, University of ...
    source:       https://github.com/SalishSeaCast/analysis-doug/blob/main/no...
    comment:      Annual climatology HRDPS-like atmospheric forcing file for ...
    history:      2024-10-09 13:35: Created.

Encoding for netCDF4 storage:

In [6]:
encoding = {
    var: {
        "chunksizes": no_snow_ds[var].shape,
        "zlib": True,
        "complevel": 4,
        "shuffle": True,
    }
    for var in ("y", "x", "snow")
}
encoding["snow"].update({
        "_FillValue": 0,
    }
)
    
encoding

{'y': {'chunksizes': (826,), 'zlib': True, 'complevel': 4, 'shuffle': True},
 'x': {'chunksizes': (710,), 'zlib': True, 'complevel': 4, 'shuffle': True},
 'snow': {'chunksizes': (826, 710),
  'zlib': True,
  'complevel': 4,
  'shuffle': True,
  '_FillValue': 0}}

Store netCDF file:

In [7]:
no_snow_ds.to_netcdf("/data/dlatorne/MEOPAR/grid/sss150/no_snow.nc", encoding=encoding, engine="h5netcdf")